#### In this task we will work on a dataset with pandas

In [1]:
# modules we'll use
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# read in 2 datasets provided
nfl_data = "NFL Play by Play 2009-2016 (v3).csv"
pd.set_option('display.max_columns', None) # What does this option do? Answer below in a comment

# ANSWER: 
df = pd.read_csv(nfl_data)
# Shows all the columns when df is printed without wrapping.

/Users/markgazol/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,21,25,36,37,55,76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# look at a few rows of the data
df.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,Returner,BlockingPlayer,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,30.0,30.0,0,0,0.0,NaN,PIT,TEN,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,1,39,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Kickoff,NaN,None,0,NaN,NaN,0,0,0,NaN,0,NaN,NaN,None,0,NaN,NaN,NaN,None,0,NaN,S.Logan,NaN,M.Griffin,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001506,0.179749,0.006639,0.281138,0.213700,0.003592,0.313676,0.0,0.0,0.323526,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009.0,NaN,NaN,NaN
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,58.0,10,5,0.0,0.0,PIT,TEN,(14:53) B.Roethlisberger pass short left to H....,1,5,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Pass,B.Roethlisberger,00-0022924,1,Complete,Short,-3,8,0,left,0,NaN,NaN,None,0,NaN,NaN,H.Ward,00-0017162,1,NaN,NaN,NaN,C.Hope,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0,NaN,NaN,NaN
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,47.0,53.0,5,2,0.0,0.0,PIT,TEN,(14:16) W.Parker right end to PIT 44 for -3 ya...,1,-3,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Run,NaN,None,0,NaN,NaN,0,0,0,NaN,0,NaN,W.Parker,00-0022250,1,right,end,NaN,None,0,NaN,NaN,NaN,S.Tulloch,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001057,0.105106,0.000981,0.162747,0.304805,0.003826,0.421478,0.0,0.0,2.415907,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009.0,NaN,NaN,NaN
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,44.0,56.0,8,2,0.0,0.0,PIT,TEN,(13:35) (Shotgun) B.Roethlisberger pass incomp...,1,0,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Pass,B.Roethlisberger,00-0022924,1,Incomplete Pass,Deep,34,0,0,right,0,NaN,NaN,None,0,NaN,NaN,M.Wallace,00-0026901,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001434,0.149088,0.001944,0.234801,0.289336,0.004776,0.318621,0.0,0.0,1.013147,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0,NaN,NaN,NaN
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,44.0,56.0,8,2,0.0,1.0,PIT,TEN,(13:27) (Punt formation) D.Sepulveda punts 54 ...,1,0,0,0,NaN,NaN,NaN,0.0,0.0,Clean,Punt,NaN,None,0,NaN,NaN,0,0,0,NaN,0,NaN,NaN,None,0,NaN,NaN,NaN,None,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001861,0.213480,0.003279,0.322262,0.244603,0.006404,0.208111,0.0,0.0,-0.699436,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.44107

In [3]:
# get the number of missing data points per column
missing_values_count = df.isnull().sum()
print(missing_values_count[0:10])

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64


In [4]:
# how many total missing values do we have?
total_missing = df.isnull().sum().sum()

# percent of data that is missing
num_total_cells = df.size
percent = "{:.2f}%".format(total_missing/num_total_cells * 100)
percent

'26.95%'

In [5]:
# remove all columns with at least one missing value
columns_with_na_dropped = df.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,Touchdown,QBHit,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0


In [6]:
# just how much data did we lose? 
print("Columns in original dataset: %d \n" % df.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 105 

Columns with na's dropped: 23


In [7]:
# replace all NA's with 0
filled = df.fillna(0)
filled.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,Returner,BlockingPlayer,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,2009-09-10,2009091000,1,1,0.0,15:00,15,3600.0,0.0,TEN,30.0,30.0,0,0,0.0,0.0,PIT,TEN,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,1,39,0,0,0,0,0,0.0,0.0,0,Kickoff,0,None,0,0,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,0,S.Logan,0,M.Griffin,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001506,0.179749,0.006639,0.281138,0.213700,0.003592,0.313676,0.0,0.0,0.323526,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009.0,0.0,0.0,0.0
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,58.0,10,5,0.0,0.0,PIT,TEN,(14:53) B.Roethlisberger pass short left to H....,1,5,0,0,0,0,0,0.0,0.0,0,Pass,B.Roethlisberger,00-0022924,1,Complete,Short,-3,8,0,left,0,0,0,None,0,0,0,H.Ward,00-0017162,1,0,0,0,C.Hope,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0,0.0,0.0,0.0
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,47.0,53.0,5,2,0.0,0.0,PIT,TEN,(14:16) W.Parker right end to PIT 44 for -3 ya...,1,-3,0,0,0,0,0,0.0,0.0,0,Run,0,None,0,0,0,0,0,0,0,0,0,W.Parker,00-0022250,1,right,end,0,None,0,0,0,0,S.Tulloch,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001057,0.105106,0.000981,0.162747,0.304805,0.003826,0.421478,0.0,0.0,2.415907,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009.0,0.0,0.0,0.0
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,44.0,56.0,8,2,0.0,0.0,PIT,TEN,(13:35) (Shotgun) B.Roethlisberger pass incomp...,1,0,0,0,0,0,0,0.0,0.0,0,Pass,B.Roethlisberger,00-0022924,1,Incomplete Pass,Deep,34,0,0,right,0,0,0,None,0,0,0,M.Wallace,00-0026901,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001434,0.149088,0.001944,0.234801,0.289336,0.004776,0.318621,0.0,0.0,1.013147,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0,0.0,0.0,0.0
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,44.0,56.0,8,2,0.0,1.0,PIT,TEN,(13:27) (Punt formation) D.Sepulveda punts 54 ...,1,0,0,0,0,0,0,0.0,0.0,Clean,Punt,0,None,0,0,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001861,0.213480,0.003279,0.322262,0.244603,0.006404,0.208111,0.0,0.0,-0.699436,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009.0,0.0,0.0,0.0


In [8]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
bfilled = df.fillna(method='bfill').fillna(0)
bfilled.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,Returner,BlockingPlayer,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,2009-09-10,2009091000,1,1,1.0,15:00,15,3600.0,0.0,TEN,30.0,30.0,0,0,0.0,0.0,PIT,TEN,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,1,39,0,0,0,Failure,0,0.0,0.0,Clean,Kickoff,B.Roethlisberger,None,0,Complete,Short,0,0,0,left,0,T.Polamalu,W.Parker,None,0,right,end,H.Ward,None,0,Fair Catch,S.Logan,J.Ringer,M.Griffin,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001506,0.179749,0.006639,0.281138,0.213700,0.003592,0.313676,0.0,0.0,0.323526,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009.0,2009.0,2009.0,2012.0
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,58.0,10,5,0.0,0.0,PIT,TEN,(14:53) B.Roethlisberger pass short left to H....,1,5,0,0,0,Failure,0,0.0,0.0,Clean,Pass,B.Roethlisberger,00-0022924,1,Complete,Short,-3,8,0,left,0,T.Polamalu,W.Parker,None,0,right,end,H.Ward,00-0017162,1,Fair Catch,S.Logan,J.Ringer,C.Hope,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0,2009.0,2009.0,2012.0
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,47.0,53.0,5,2,0.0,0.0,PIT,TEN,(14:16) W.Parker right end to PIT 44 for -3 ya...,1,-3,0,0,0,Failure,0,0.0,0.0,Clean,Run,B.Roethlisberger,None,0,Incomplete Pass,Deep,0,0,0,right,0,T.Polamalu,W.Parker,00-0022250,1,right,end,M.Wallace,None,0,Fair Catch,S.Logan,J.Ringer,S.Tulloch,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001057,0.105106,0.000981,0.162747,0.304805,0.003826,0.421478,0.0,0.0,2.415907,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009.0,2009.0,2009.0,2012.0
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,44.0,56.0,8,2,0.0,0.0,PIT,TEN,(13:35) (Shotgun) B.Roethlisberger pass incomp...,1,0,0,0,0,Failure,0,0.0,0.0,Clean,Pass,B.Roethlisberger,00-0022924,1,Incomplete Pass,Deep,34,0,0,right,0,T.Polamalu,C.Johnson,None,0,middle,end,M.Wallace,00-0026901,0,Fair Catch,S.Logan,J.Ringer,B.Keisel,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001434,0.149088,0.001944,0.234801,0.289336,0.004776,0.318621,0.0,0.0,1.013147,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0,2009.0,2009.0,2012.0
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,44.0,56.0,8,2,0.0,1.0,PIT,TEN,(13:27) (Punt formation) 

In [9]:
# Create indicator column for missing values in data nlf_data for the column 'down'
df["down_na_indicator"] = df["down"].map(np.isnan)
df[["down_na_indicator", "down"]].head()

,down_na_indicator,down
0,True,NaN
1,False,1.0
2,False,2.0
3,False,3.0
4,False,4.0


In [10]:
# print columns that have more than 50 missing values in data nlf_data
missing_values_count[missing_values_count>50].index

Index(['down', 'time', 'TimeSecs', 'PlayTimeDiff', 'SideofField', 'yrdln',
       'yrdline100', 'GoalToGo', 'FirstDown', 'posteam', 'DefensiveTeam',
       'ExPointResult', 'TwoPointConv', 'DefTwoPoint', 'Safety', 'PuntResult',
       'PlayType', 'Passer', 'Passer_ID', 'PassOutcome', 'PassLength',
       'AirYards', 'PassLocation', 'InterceptionThrown', 'Interceptor',
       'Rusher', 'Rusher_ID', 'RunLocation', 'RunGap', 'Receiver',
       'Receiver_ID', 'ReturnResult', 'Returner', 'BlockingPlayer', 'Tackler1',
       'Tackler2', 'FieldGoalResult', 'FieldGoalDistance', 'Fumble',
       'RecFumbTeam', 'RecFumbPlayer', 'Sack', 'ChalReplayResult',
       'Accepted.Penalty', 'PenalizedTeam', 'PenaltyType', 'PenalizedPlayer',
       'Penalty.Yards', 'PosTeamScore', 'DefTeamScore', 'ScoreDiff',
       'AbsScoreDiff', 'No_Score_Prob', 'Opp_Field_Goal_Prob',
       'Opp_Safety_Prob', 'Opp_Touchdown_Prob', 'Field_Goal_Prob',
       'Safety_Prob', 'Touchdown_Prob', 'ExpPts', 'EPA', 'airEPA', 'y

In [11]:
# What are the data types for each column?
df.dtypes

Date                  object
GameID                 int64
Drive                  int64
qtr                    int64
down                 float64
                      ...   
Season               float64
Unnamed: 102         float64
Unnamed: 103         float64
Unnamed: 104         float64
down_na_indicator       bool
Length: 106, dtype: object

In [12]:
# What are the column names in the data?
df.columns.tolist()

['Date',
 'GameID',
 'Drive',
 'qtr',
 'down',
 'time',
 'TimeUnder',
 'TimeSecs',
 'PlayTimeDiff',
 'SideofField',
 'yrdln',
 'yrdline100',
 'ydstogo',
 'ydsnet',
 'GoalToGo',
 'FirstDown',
 'posteam',
 'DefensiveTeam',
 'desc',
 'PlayAttempted',
 'Yards.Gained',
 'sp',
 'Touchdown',
 'ExPointResult',
 'TwoPointConv',
 'DefTwoPoint',
 'Safety',
 'Onsidekick',
 'PuntResult',
 'PlayType',
 'Passer',
 'Passer_ID',
 'PassAttempt',
 'PassOutcome',
 'PassLength',
 'AirYards',
 'YardsAfterCatch',
 'QBHit',
 'PassLocation',
 'InterceptionThrown',
 'Interceptor',
 'Rusher',
 'Rusher_ID',
 'RushAttempt',
 'RunLocation',
 'RunGap',
 'Receiver',
 'Receiver_ID',
 'Reception',
 'ReturnResult',
 'Returner',
 'BlockingPlayer',
 'Tackler1',
 'Tackler2',
 'FieldGoalResult',
 'FieldGoalDistance',
 'Fumble',
 'RecFumbTeam',
 'RecFumbPlayer',
 'Sack',
 'Challenge.Replay',
 'ChalReplayResult',
 'Accepted.Penalty',
 'PenalizedTeam',
 'PenaltyType',
 'PenalizedPlayer',
 'Penalty.Yards',
 'PosTeamScore',
 'De

In [13]:
a=df.select_dtypes("object").nunique()

In [14]:
text_cols_to_drop = a[a>50].index.tolist()

In [15]:
# text_cols_to_drop = list(set(text_cols_to_drop) - {"PlayType"})

In [16]:
df1 = df.drop(text_cols_to_drop,axis=1)

In [17]:
for col in df1.dtypes[df1.dtypes=="object"].index:
#     if col != "PlayType":
    df1 = df1.join(pd.get_dummies(df1[col]).iloc[:,:-1], rsuffix=f"_{col}")
    df1 = df1.drop(col, axis=1)

In [18]:
df1 = df1.dropna(subset=['GoalToGo'])
df_dropped = df1.dropna(axis=1)


In [19]:
# Split the data into test and train
train, test = train_test_split(df_dropped, test_size=0.3, random_state=1)

In [20]:
# How many rows in the train set?
train.shape[0]

253206

In [21]:
# Import RandomForest model and fit to the nld_data data to predict the column PlayType. 
target_col = "GoalToGo"
Y_train = train[target_col]
X_train = train.drop(target_col, axis=1)
Y_test = test[target_col]
X_test = test.drop(target_col, axis=1)


# You can choose another column if you want.
model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [22]:
# Predict 
prediciton_train = model.predict_proba(X_train)
prediciton_test = model.predict_proba(X_test)

In [23]:
# Evaluate
not_filled_roc_auc = roc_auc_score(Y_test, prediciton_test[:,-1])

In [24]:
# Retrain the model on datasets filled and bfilled

# filled
filled = filled.drop(text_cols_to_drop, axis=1)
for col in filled.dtypes[filled.dtypes=="object"].index:
#     if col != "PlayType":
    filled = filled.join(pd.get_dummies(filled[col]).iloc[:,:-1], rsuffix=f"_{col}")
    filled = filled.drop(col, axis=1)
    
filled = filled.dropna(subset=['GoalToGo'])
df_dropped = filled.dropna(axis=1)

train_filled, test_filled = train_test_split(df_dropped, test_size=0.3, random_state=1)


target_col = "GoalToGo"
Y_train_filled = train_filled[target_col]
X_train_filled = train_filled.drop(target_col, axis=1)
Y_test_filled = test_filled[target_col]
X_test_filled = test_filled.drop(target_col, axis=1)


model_filled = RandomForestClassifier(random_state=42)
model_filled.fit(X_train_filled, Y_train_filled)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [25]:
# Predict 
prediciton_test_filled = model_filled.predict_proba(X_test_filled)

In [26]:
# Evaluate
filled_roc_auc = roc_auc_score(Y_test_filled, prediciton_test_filled[:,-1])

In [27]:
# bfilled
bfilled = bfilled.drop(text_cols_to_drop, axis=1)
for col in bfilled.dtypes[bfilled.dtypes=="object"].index:
#     if col != "PlayType":
    bfilled = bfilled.join(pd.get_dummies(bfilled[col]).iloc[:,:-1], rsuffix=f"_{col}")
    bfilled = bfilled.drop(col, axis=1)
    
bfilled = bfilled.dropna(subset=['GoalToGo'])
df_dropped = bfilled.dropna(axis=1)

train_bfilled, test_bfilled = train_test_split(df_dropped, test_size=0.3, random_state=1)


target_col = "GoalToGo"
Y_train_bfilled = train_bfilled[target_col]
X_train_bfilled = train_bfilled.drop(target_col, axis=1)
Y_test_bfilled = test_bfilled[target_col]
X_test_bfilled = test_bfilled.drop(target_col, axis=1)


model_bfilled = RandomForestClassifier(random_state=42)
model_bfilled.fit(X_train_bfilled, Y_train_bfilled)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [28]:
# Predict 
prediciton_test_bfilled = model_bfilled.predict_proba(X_test_bfilled)

In [29]:
# Evaluate
bfilled_roc_auc = roc_auc_score(Y_test_bfilled, prediciton_test_bfilled[:,-1])

In [31]:
# Compare results
print("filled vs bfilled")
print("roc_auc_difference:", filled_roc_auc-bfilled_roc_auc)

filled vs bfilled
roc_auc_difference: 5.603343253735016e-05
